In [1]:
import numpy as np
import mnist_loader as ml

In [2]:
training_data, validation_data, test_data = ml.load_data_wrapper()

In [19]:
class Layer:
    
    def __init__(self, weights):
        
        self.FPoutput = None
        self.BPoutput = None
        self.weights = weights
        
    def forwardPass(self, inputData):
        
        input = np.vstack(([1], inputData))
        net = self.weights @ input
        self.FPoutput = self.activation(net)
        
    def activation(self, x):
        
        return 1/(1+np.exp(-x))

    #######################################################################
    # for last layer, the inputData = self.FPoutput - trueLabel
    # for each next layer, the inputData = self.BPoutput from PREVIOUS LAYER
    #######################################################################
    
    def backwardPropagation(self, inputData, lr):
        
        delta = inputData * (1 - self.FPoutput)
        dLdW = delta @ self.input.T
        new_weights = self.weights - lr * dLdW
        dLdX = self.weights.T @ delta
        self.weights = new_weights
        self.BPoutput = dLdX

In [27]:
w1 = np.array([[0.1, -0.2, 0.3], [-0.4, 0.5, -0.6]])
w2 = np.array([[0.15, -0.25, 0.35], [-0.45, 0.55, -0.65]])

In [28]:
L1 = Layer(w1)
L2 = Layer(w2)

In [29]:
inp1 = np.array([[1], [0]])
L1.forwardPass(inp1)
inp2 = L1.FPoutput
L2.forwardPass(inp2)
out = L2.FPoutput
out

array([[0.55354082],
       [0.37052271]])

In [34]:
inp2b = out - np.array([[1, 0]]).T
inp2b

array([[-0.44645918],
       [ 0.37052271]])

In [83]:
out

array([[0.45387836],
       [0.61538967]])

In [7]:
w2b = np.array([[-0.110335, -0.0524114, -0.0579236], [0.0864191, 0.0410509, 0.0453682]])

In [14]:
delta = np.array([-0.110334967, 0.086419099])
delta.shape

(2,)

In [15]:
w2.T @ delta

array([-0.05543884,  0.07511425, -0.09478965])

In [4]:
a, b = zip(*training_data)

In [22]:
len(a[0])

784

In [5]:
b[0]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [24]:
test = a[0]

In [28]:
test2 = np.vstack(([1],test))

In [31]:
len(test2)

785

In [32]:
np.array([0.1]*10)

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [39]:
1/(1+np.exp(0.7))

0.3318122278318339

In [40]:
def activation(x):

    return 1/(1+np.exp(x))

In [42]:
activation(b[0])

array([[0.5       ],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.26894142],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.5       ]])

In [43]:
b[0]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [46]:
a[0].shape

(784, 1)

In [47]:
w = np.random.uniform(size=(10, 784))

In [48]:
w.shape

(10, 784)

In [50]:
c = w @ a[0]

In [51]:
c - b[0]

array([[53.8964059 ],
       [55.59064082],
       [57.26543014],
       [55.39124241],
       [55.97163913],
       [50.24210433],
       [51.49004979],
       [59.19191975],
       [54.17061921],
       [53.91409691]])

In [56]:
np.array([1, 2]) * np.array([3, 4])

array([3, 8])